<div style="float: left;">
C. Dossal, P. Noble
</div>
<div style="float: right;">
Novembre 2017
</div>
<br>

<div align="center"; style="font-size: 200%; font-weight: bold;">
TP Vocoder de Phase
</div>

In [1]:
import os
import sys
#sys.path.append('/Users/pascal/Documents/ENSEIGNEMENTS/INSA_4A/AF-SIGNAL-2016/FichiersTP2Signal')
#mettre le chemin où vous allez récupérer vos modules.
%matplotlib inline
from wavtools import *
import numpy as np

In [2]:
def calc_hanning(m,n):
    return .5*(1 - np.cos(2*np.pi*np.linspace(1,m,m)/(n+1)))                 

In [3]:
def hanning(n):
    if (n%2) == 0:
        # Even length window
        half = n//2
        w = calc_hanning(half,n)
        w = np.concatenate((w,w[::-1]))
    else:
        # Odd length window
        half = (n+1)//2
        w = calc_hanning(half,n)
        z=w[::-1]
        w = np.concatenate((w,z[1:]))
    return w

In [4]:
def Analyse(S,N):
    NS=S.shape[0]
    Nf=8*NS//N-7
    Spec=np.zeros((N,Nf))
    Phase=np.zeros((N,Nf))
    H=hanning(N);
    for i in range(Nf):
        temp=H*S[i*N//8:(i+8)*N//8]
        ftemp=np.fft.fft(temp)
        Spec[:,i]=np.abs(ftemp)
        Phase[:,i]=np.angle(ftemp)
    return Spec,Phase

In [5]:
def InterpSpec(Spec,T):
    N=Spec.shape[0]
    Nf2=T.shape[0]
    Spec2=np.zeros((N,Nf2))
    for k in range(Nf2):
        t=T[k]
        indice1=int(np.floor(t))
        alpha=t-indice1
        Spec2[:,k]=(1-alpha)*Spec[:,indice1]+alpha*Spec[:,indice1+1]
    return Spec2

In [6]:
def Synthese(Spec,Phase):
    N=Spec.shape[0]
    Nf=Spec.shape[1]
    Srec=np.zeros((Nf+7)*N//8)
    H=hanning(N)
    for k in range(Nf):
        ftemp=Spec[:,k]*np.exp(1j*Phase[:,k])
        temp=np.real(np.fft.ifft(ftemp))
        Srec[k*N//8:(k+8)*N//8]=Srec[k*N//8:(k+8)*N//8]+H*temp
    Srec=Srec/4
    return Srec

In [7]:
def InterpPhase(Phase,T):
    N=Phase.shape[0]
    Nf2=T.shape[0]
    Phase2=np.zeros((N,Nf2))
    t0=T[0]
    indice1=int(np.floor(t0))
    Phase2[:,0]=Phase[:,indice1]
    for k in range(1,Nf2):
        t=T[k]
        indice1=int(np.floor(t))
        dphase=Phase[:,indice1+1]-Phase[:,indice1]
        Phase2[:,k]=Phase2[:,k-1]+dphase
    return Phase2

In [8]:
#S, Fs = open_wave('/Users/pascal/Documents/ENSEIGNEMENTS/INSA_4A/AF-SIGNAL-2016/FichiersTP2Signal/bird.wav')
S, Fs = open_wave('samples/bird.wav')

In [9]:
N=1024
Spec,Phase=Analyse(S,N)

In [10]:
Nf=Spec.shape[1]
alpha=2
T=np.arange(0,Nf-3,alpha)

In [11]:
Spec2=InterpSpec(Spec,T)

In [12]:
Phase2=InterpPhase(Phase,T)

In [13]:
Srec=Synthese(Spec2,Phase2)

In [14]:
save_wave(Srec, 'essaiPN.wav', Fs)

In [15]:
def vocoder(alpha):
    #S, Fs = open_wave('/Users/pascal/Documents/ENSEIGNEMENTS/INSA_4A/AF-SIGNAL-2016/FichiersTP2Signal/bird.wav')
    S, Fs = open_wave('samples/bird.wav')
    N=1024
    Spec,Phase=Analyse(S,N)
    Nf=Spec.shape[1]
    T=np.arange(0,Nf-3,alpha)
    Spec2=InterpSpec(Spec,T)
    Phase2=InterpPhase(Phase,T)
    Srec=Synthese(Spec2,Phase2)
    #save_wave(Srec, '/Users/pascal/Documents/ENSEIGNEMENTS/INSA_4A/AF-SIGNAL-2016/FichiersTP2Signal/essaiPN.wav', Fs)
    save_wave(Srec, 'essaiPN.wav', Fs)

In [16]:
vocoder(0.5)